In [1]:
import stim
import numpy as np
from node_qubit import Node, Pauli
from rgs import HalfRGS, RGS
from rgs_probability_calculation import prob_rgs_trial
from tree_code_helper import tree_code_physical_measure

# Repeater Graph State Bell pair distribution protocol

[The protocol implemented in this paper is based on https://arxiv.org/abs/2306.03748]

This notebook implements the two-stage protocol for all-photonic quantum repeater based on repeater graph state (RGS) originally proposed by [Azuma et al., [NatComm. 2015, 6:6787]](https://doi.org/10.1038/ncomms7787).
Each step of the protocol is implemented in its own function where all the steps are annotated in `experiment_setup()` function.

To verify the protocol, we check the final stabilizer generators of the quantum state.
The protocol is correct if the states shared between two ends (alice and bob) are described with stabilizer Z_alice X_bob and Z_bob X_alice.

There is no simulation of classical communication between nodes but when a step requires classical communication, the information will be stored and processed with other information in a different step. This is to ensure that we cannot cheat by having extra information and to agree with our proposed protocol.

In [2]:
def measurements_at_absa(t: stim.TableauSimulator, m: int, left_halfs: list[Node], right_halfs: list[Node]) -> int:
    """measurement of all qubits in the RGS (step 1) and return the index of the arm that has a successful BSM"""
    if m != len(left_halfs) or m != len(right_halfs):
        ValueError(f"number of arms {m} does not equal the input length of list of two halves {len(left_halfs)}, {len(right_halfs)}")

    # BSM part (outer qubit measurements)
    success_arm_index = -1
    for i in range(m):
        unode = left_halfs[i]
        vnode = right_halfs[i]

        if unode.is_lost or vnode.is_lost:
            continue

        u = unode.qubit_index
        v = vnode.qubit_index
        t.cz(u, v)
        t.h(u, v)
        unode.measurement_result = unode.eigenvalue = t.measure(u)
        vnode.measurement_result = vnode.eigenvalue = t.measure(v)
        unode.measurement_basis = vnode.measurement_basis = Pauli.X

        # simulating linear optics; consider +1/-1 and -1/+1 to be the two case ABSAs can distinguish
        if success_arm_index == -1 and (unode.measurement_result != vnode.measurement_result):
            success_arm_index = i

    # inner qubits measurements
    for i in range(m):
        unode = left_halfs[i]
        vnode = right_halfs[i]
        if i == success_arm_index:
            tree_code_physical_measure(t, unode, Pauli.X)
            tree_code_physical_measure(t, vnode, Pauli.X)
        else:
            tree_code_physical_measure(t, unode, Pauli.Z)
            tree_code_physical_measure(t, vnode, Pauli.Z)

    return success_arm_index


def update_tree_with_outer_qubits(left_tree_root: Node, right_tree_root: Node):
    """update in place with the BSM results; toggling 1st level results with root of another tree (step 2)"""
    if left_tree_root.is_lost or right_tree_root.is_lost:
        RuntimeError("trying to update tree with outer qubits that were lost!")

    if left_tree_root.eigenvalue:
        for u in right_tree_root.children:
            if u.is_lost:
                continue
            u.eigenvalue = not u.eigenvalue
    if right_tree_root.eigenvalue:
        for u in left_tree_root.children:
            if u.is_lost:
                continue
            u.eigenvalue = not u.eigenvalue


def compute_parity_for_end_nodes(m: int, left_logical_results: list[bool], right_logical_results: list[bool], successful_bsm_index: int) -> tuple[bool, bool]:
    """apply the parity at end nodes (step 3)
    return tuple of parity to be sent to the left and right respectively"""
    # parity multiplied together of Z of left (right) is sent to right (left),
    # while X of left (right) is sent to left (right).
    left_par, right_par = False, False
    for i in range(m):
        if i == successful_bsm_index:
            continue
        left_par ^= left_logical_results[i]
        right_par ^= right_logical_results[i]
    left_par ^= right_logical_results[successful_bsm_index]
    right_par ^= left_logical_results[successful_bsm_index]
    return left_par, right_par

In [3]:
total_photons = 0
lost_photons = 0

rng = np.random.default_rng()


def experiment_setup(
    number_of_hops: int,
    m: int,
    branching_parameters: list[int],
    loss_probability: float = 0,
    # photon_error_probability: float = 0,
    # emitter_error_probability: float = 0,
) -> tuple[bool, int, int]:
    """One run of the biclique RGS protocol
    Return: success-or-failure of the trial (bool), expectation value of ZX, expectation value of XZ at the end between Alice and Bob"""
    global total_photons, lost_photons
    # Ancilla qubits we require (total 4)
    #   temporary anchor for tree encoding: 1 (ancilla[0])
    #   emitter for outer qubit: 1 (ancilla[1])
    #   anchor for the half-RGS: 2 (ancilla[2-3])
    # qubit range
    alice = 0
    bob = 1
    # ancilla qubits
    anchor_left = 2
    anchor_right = 3
    outer_emitter = 4
    root_id = 5
    # starting index of unused qubit
    next_id = 6

    # we need (hop - 1) RGS
    rgss = [RGS(m, branching_parameters) for _ in range(number_of_hops - 1)]
    half_alice = HalfRGS(m, branching_parameters, alice)
    half_bob = HalfRGS(m, branching_parameters, bob)

    # assign qubit indices
    for rgs in rgss:
        next_id = rgs.assign_qubit_indices(next_id)
    next_id = half_alice.assign_qubit_indices(next_id)
    next_id = half_bob.assign_qubit_indices(next_id)

    # RGS creation
    t = stim.TableauSimulator()
    for rgs in rgss:
        rgs.initialize_quantum_state(t, anchor_left, anchor_right, outer_emitter, root_id)
    half_alice.initialize_quantum_state(t, outer_emitter, root_id)
    half_bob.initialize_quantum_state(t, outer_emitter, root_id)

    # process photon loss
    for rgs in rgss:
        rgs.process_photon_loss(t, loss_probability, rng)
    half_alice.process_photon_loss(t, loss_probability, rng)
    half_bob.process_photon_loss(t, loss_probability, rng)

    # Debugging, check how many photon got lost
    for rgs in rgss:
        lost_ph, total_ph = rgs.count_lost_photons()
        lost_photons += lost_ph
        total_photons += total_ph
    lost_ph, total_ph = half_alice.count_lost_photons()
    lost_photons += lost_ph
    total_photons += total_ph
    lost_ph, total_ph = half_bob.count_lost_photons()
    lost_photons += lost_ph
    total_photons += total_ph

    # (Protocol step 1) ABSA measurements
    success_bsm_indices = [-1] * number_of_hops  # number of ABSAs in the repeater chain
    for i in range(len(rgss) - 1):
        success_bsm_indices[i + 1] = measurements_at_absa(t, m, rgss[i].right_arms, rgss[i + 1].left_arms)
        rgss[i].successful_right_arm_index = rgss[i + 1].successful_left_arm_index = success_bsm_indices[i + 1]
    if len(rgss) > 0:
        success_bsm_indices[0] = measurements_at_absa(t, m, half_alice.arms, rgss[0].left_arms)
        success_bsm_indices[-1] = measurements_at_absa(t, m, rgss[-1].right_arms, half_bob.arms)
        rgss[0].successful_left_arm_index = half_alice.successful_arm_index = success_bsm_indices[0]
        rgss[-1].successful_right_arm_index = half_bob.successful_arm_index = success_bsm_indices[-1]
    else:
        # special case for 1 hop (no RGSS source nodes)
        success_bsm_indices[0] = measurements_at_absa(t, m, half_alice.arms, half_bob.arms)
        half_alice.successful_arm_index = half_bob.successful_arm_index = success_bsm_indices[0]

    # print(success_bsm_indices)
    if any(map(lambda id: id == -1, success_bsm_indices)):
        return False, None, None, None, None

    # (Protocol Step 1) Update measurements tree by assigning eigenvalues to the nodes taking side effects into account
    # imitating the classical messages received from RGSSs to ABSAs
    # we need to take note of the successful BSM arm index to denote the arm that undergone logical X measurements of inner qubits
    half_alice.update_measurement_with_side_effects()
    half_bob.update_measurement_with_side_effects()
    for i, rgs in enumerate(rgss):
        rgs.update_measurements_with_side_effect()

    # (Protocol Step 2) Propagating side effects of BSMs of outer qubits into their connected inner qubits
    bsm_arm_pairs = [half_alice.arms[success_bsm_indices[0]]]
    for i in range(len(rgss)):
        bsm_arm_pairs.append(rgss[i].left_arms[success_bsm_indices[i]])
        bsm_arm_pairs.append(rgss[i].right_arms[success_bsm_indices[i + 1]])
    bsm_arm_pairs.append(half_bob.arms[success_bsm_indices[-1]])
    for i in range(0, len(bsm_arm_pairs), 2):
        update_tree_with_outer_qubits(bsm_arm_pairs[i], bsm_arm_pairs[i + 1])

    # (Protocol Step 2/3?) Decoding logical measurements
    is_trial_successful = half_alice.decode_logical_results()
    is_trial_successful &= half_bob.decode_logical_results()
    for rgs in rgss:
        is_trial_successful &= rgs.decode_logical_results()
    if not is_trial_successful:
        return False, None, None, None, None

    # (Protocol Step 3) Compute parity at each ABSA for Pauli frame corrections
    parities: list[tuple[bool, bool]] = []
    if number_of_hops == 1:
        parities.append(compute_parity_for_end_nodes(m, half_alice.logical_results, half_bob.logical_results, success_bsm_indices[0]))
    else:
        parities.append(compute_parity_for_end_nodes(m, half_alice.logical_results, rgss[0].left_logical_results, success_bsm_indices[0]))
        for i in range(len(rgss) - 1):
            parities.append(compute_parity_for_end_nodes(m, rgss[i].right_logical_results, rgss[i + 1].left_logical_results, success_bsm_indices[i + 1]))
        parities.append(compute_parity_for_end_nodes(m, rgss[-1].right_logical_results, half_bob.logical_results, success_bsm_indices[-1]))

    # (Protocol Step 4) Combining all the parities from all ABSAs and correct at end nodes
    total_parity = (False, False)
    for l, r in parities:
        total_parity = total_parity[0] ^ l, total_parity[1] ^ r
    if total_parity[0]:
        t.z(alice)
    if total_parity[1]:
        t.z(bob)

    return True, t.peek_observable_expectation(stim.PauliString("XZ")), t.peek_observable_expectation(stim.PauliString("ZX")), t.canonical_stabilizers(), total_parity

In [4]:
def rgs_trial_experiment_run(required_runs: int, number_of_hops: int, m: int, bv: int, photon_loss_probability: float, show_progress_mark=False):
    global total_photons, lost_photons
    num_ticks = 10
    progress_marks = [int(i * required_runs / num_ticks) for i in range(1, num_ticks)]
    progress_marks.append(required_runs)

    fail_count = 0
    good_count = 0
    actual_run_count = 0

    total_photons = 0
    lost_photons = 0

    print(f'start RGS protocol trials for {number_of_hops} hops with photon loss probability between RGSS and ABSA {photon_loss_probability}.')
    print(f'      RGS parameters are given by m = {m} and branching parameters {bv}.')
    print(f'============================')

    while actual_run_count < required_runs:
        is_successful, exp_xz, exp_zx, cano_stabs, parity = experiment_setup(number_of_hops, m, bv, photon_loss_probability)
        if is_successful and (exp_xz != 1 or exp_zx!= 1):
            fail_count += 1
        elif is_successful and exp_zx == 1 and exp_xz == 1:
            good_count += 1
        actual_run_count += 1
        # progress print
        if show_progress_mark and actual_run_count in progress_marks:
            print(f'    has been running for {actual_run_count} trials with {good_count} successful distribution(s).')

    print(f'photon accounting loss_rate = {lost_photons/total_photons} ({lost_photons}, {total_photons})')
    print(f'fail count = {fail_count}, good count = {good_count}')
    print(f'fail ratio = {fail_count/(fail_count + good_count)}')
    print('----------------------')
    print(f"theoretical success probability is {prob_rgs_trial(m, bv, (1-photon_loss_probability), number_of_hops)}")
    print(f'success probability = {(fail_count + good_count) / actual_run_count}')
    print('----------------------')
    print(f'ran {actual_run_count} times')

In [5]:
rgs_trial_experiment_run(100, 1, 14, [10, 5], 0.1, True)

start RGS protocol trials for 1 hops with photon loss probability between RGSS and ABSA 0.1.
      RGS parameters are given by m = 14 and branching parameters [10, 5].
    has been running for 10 trials with 10 successful distribution(s).
    has been running for 20 trials with 20 successful distribution(s).
    has been running for 30 trials with 30 successful distribution(s).
    has been running for 40 trials with 40 successful distribution(s).
    has been running for 50 trials with 50 successful distribution(s).
    has been running for 60 trials with 60 successful distribution(s).
    has been running for 70 trials with 70 successful distribution(s).
    has been running for 80 trials with 80 successful distribution(s).
    has been running for 90 trials with 90 successful distribution(s).
    has been running for 100 trials with 100 successful distribution(s).
photon accounting loss_rate = 0.10053864168618266 (17172, 170800)
fail count = 0, good count = 100
fail ratio = 0.0
----

In [8]:
# 1-hop cases; no loss
rgs_trial_experiment_run(1000000, 1, 1, [1], 0, True)

start RGS protocol trials for 1 hops with photon loss probability between RGSS and ABSA 0.
      RGS parameters are given by m = 1 and branching parameters [1].
    has been running for 100000 trials with 50308 successful distribution(s).
    has been running for 200000 trials with 100260 successful distribution(s).
    has been running for 300000 trials with 150472 successful distribution(s).
    has been running for 400000 trials with 200445 successful distribution(s).
    has been running for 500000 trials with 250506 successful distribution(s).
    has been running for 600000 trials with 300793 successful distribution(s).
    has been running for 700000 trials with 350941 successful distribution(s).
    has been running for 800000 trials with 401160 successful distribution(s).
    has been running for 900000 trials with 451191 successful distribution(s).
    has been running for 1000000 trials with 501081 successful distribution(s).
photon accounting loss_rate = 0.0 (0, 4000000)
fa

In [6]:
# with more than one arm and with lossy links
# at loss_probability = 0.1, this means that the distance between RGSS and the ABSA is ~2.28787km
rgs_trial_experiment_run(10000, 1, 3, [1], 0.1, True)

start RGS protocol trials for 1 hops with photon loss probability between RGSS and ABSA 0.1.
      RGS parameters are given by m = 3 and branching parameters [1].
    has been running for 1000 trials with 422 successful distribution(s).
    has been running for 2000 trials with 837 successful distribution(s).
    has been running for 3000 trials with 1259 successful distribution(s).
    has been running for 4000 trials with 1663 successful distribution(s).
    has been running for 5000 trials with 2083 successful distribution(s).
    has been running for 6000 trials with 2479 successful distribution(s).
    has been running for 7000 trials with 2911 successful distribution(s).
    has been running for 8000 trials with 3313 successful distribution(s).
    has been running for 9000 trials with 3739 successful distribution(s).
    has been running for 10000 trials with 4159 successful distribution(s).
photon accounting loss_rate = 0.09979166666666667 (11975, 120000)
fail count = 0, good c

In [7]:
# with logical qubits
rgs_trial_experiment_run(10000, 1, 1, [10], 0.1, True)

start RGS protocol trials for 1 hops with photon loss probability between RGSS and ABSA 0.1.
      RGS parameters are given by m = 1 and branching parameters [10].
    has been running for 1000 trials with 424 successful distribution(s).
    has been running for 2000 trials with 847 successful distribution(s).
    has been running for 3000 trials with 1255 successful distribution(s).
    has been running for 4000 trials with 1636 successful distribution(s).
    has been running for 5000 trials with 2015 successful distribution(s).
    has been running for 6000 trials with 2392 successful distribution(s).
    has been running for 7000 trials with 2801 successful distribution(s).
    has been running for 8000 trials with 3190 successful distribution(s).
    has been running for 9000 trials with 3591 successful distribution(s).
    has been running for 10000 trials with 4026 successful distribution(s).
photon accounting loss_rate = 0.09943636363636364 (21876, 220000)
fail count = 0, good 

In [12]:
# with logical qubits
rgs_trial_experiment_run(10000, 1, 3, [1], 0.1, True)

start RGS protocol trials for 1 hops with photon loss probability between RGSS and ABSA 0.1.
      RGS parameters are given by m = 3 and branching parameters [1].
    has been running for 1000 trials with 416 successful distribution(s).
    has been running for 2000 trials with 822 successful distribution(s).
    has been running for 3000 trials with 1248 successful distribution(s).
    has been running for 4000 trials with 1681 successful distribution(s).
    has been running for 5000 trials with 2122 successful distribution(s).
    has been running for 6000 trials with 2550 successful distribution(s).
    has been running for 7000 trials with 2980 successful distribution(s).
    has been running for 8000 trials with 3388 successful distribution(s).
    has been running for 9000 trials with 3785 successful distribution(s).
    has been running for 10000 trials with 4203 successful distribution(s).
photon accounting loss_rate = 0.10121666666666666 (12146, 120000)
fail count = 0, good c

In [11]:
rgs_trial_experiment_run(10000, 3, 5, [5, 3], 0.1, True)

start RGS protocol trials for 3 hops with photon loss probability between RGSS and ABSA 0.1.
      RGS parameters are given by m = 5 and branching parameters [5, 3].
    has been running for 1000 trials with 750 successful distribution(s).
    has been running for 2000 trials with 1506 successful distribution(s).
    has been running for 3000 trials with 2266 successful distribution(s).
    has been running for 4000 trials with 3017 successful distribution(s).
    has been running for 5000 trials with 3776 successful distribution(s).
    has been running for 6000 trials with 4523 successful distribution(s).
    has been running for 7000 trials with 5285 successful distribution(s).
    has been running for 8000 trials with 6063 successful distribution(s).
    has been running for 9000 trials with 6817 successful distribution(s).
    has been running for 10000 trials with 7600 successful distribution(s).
photon accounting loss_rate = 0.09997507936507936 (629843, 6300000)
fail count = 0, 